<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/mnemonics_mnemonicsfinetuning_updatemancante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'

!rm -r IncrementalLearning
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=a5bd19b266a406e14c289448eebfa772719743743f8f9309080879a124594e93
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
rm: cannot remove 'IncrementalLearning': No such file or directory
Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 555 (delta 47), reused 0 (delta 0), pack-reused 474
Receiving objects: 100% (555/555), 614.38 KiB | 12.54 MiB/s, done.
Resolving deltas: 100% (322/322), done.


In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Subset, DataLoader, Dataset
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

import import_ipynb
from IncrementalLearning.cifar100 import ilCIFAR100

from IncrementalLearning.resnet_cifar import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

importing Jupyter notebook from /content/IncrementalLearning/cifar100.ipynb
Files already downloaded and verified
importing Jupyter notebook from /content/IncrementalLearning/resnet_cifar.ipynb


In [2]:
# SOME UTILS FUNCTIONS

def tensor2im(input_image, imtype=np.uint8):
    mean = [0.5071, 0.4867, 0.4408]
    std = [0.2675, 0.2565, 0.2761]
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor.cpu().detach().float().numpy()
        if image_numpy.shape[0] == 1:
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        for i in range(len(mean)): 
            image_numpy[i] = image_numpy[i] * std[i] + mean[i]
        image_numpy = image_numpy * 255
        image_numpy = np.transpose(image_numpy, (1, 2, 0))
    else:
        image_numpy = input_image
    return image_numpy.astype(imtype)


In [28]:
class mnemonics():
  def __init__(self, randomseed):
    self.model = resnet32(num_classes=100).to('cuda')
    self.temp_model = None
    self.lr = 2
    self.gamma = 0.2
    self.weight_decay = 1e-5 
    self.milestones = [49,63]
    self.batch_size = 128
    self.numepochs = 1
    self.n_classes = 0
    self.n_known = 0
    self.feature_size=64
    self.momentum=0.9
    self.criterion = nn.BCEWithLogitsLoss()

    self.NUM_BATCHES=10
    self.randomseed=randomseed
    self.trainloader=None
    self.testloader=None
    self.CLASSES_PER_BATCH=10
    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed)
    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train=False)

    self.last_test = None
    self.y_pred = []
    self.y_test = []

    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()

    self.exemplar_features_mean = None
    # lista di liste, ogni lista contiene gli exemplars di una classe
    self.exemplar_sets_idxs = [] # mn_exemplat_sets
    # lista unica, tutti gli indici degli exemplar
    self.exemplar_idxs = []

  def basic_training(self, model, epochs, loader, optimizer, scheduler=None):
    for epoch in range(epochs):
      for _, inputs, labels in loader:
        inputs = inputs.float().to('cuda')
        labels = torch.tensor([self.diz[c.item()] for c in labels])
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        labels_encoded = F.one_hot(labels,100).float().cuda()
        loss = self.criterion(outputs,labels_encoded)
        loss.backward()
        optimizer.step()
    
      if scheduler is not None:
        scheduler.step()

  def update_params(self, 
                    finetuning_idxs, 
                    training_idxs, 
                    mnemonics_to_optimize, 
                    batch_size,
                    lr=0.01, 
                    momentum=0.9, 
                    weight_decay=1e-5, 
                    milestones=[10, 20, 30, 40],
                    gamma=0.5, 
                    tuning_epochs=1,
                    updating_epochs=5):
    
    """
    finetuning_idxs = indexes of current task elements
    mnemonics_idxs = indexes of exemplar elements
    mnemonics_to_optimize = the optimized parameters in the update phase
    """

    # make a copy of the model
    model_copy = copy.deepcopy(self.model)
    model_copy.train()
    model_copy.to('cuda')

    # define the loss
    criterion = nn.BCEWithLogitsLoss()

    # FINE TUNING FOR 1 EPOCH eq. 8 --------------------------------------------
    
    # define optimizer and scheduler for fine tuning phase
    optimizer = optim.SGD(model_copy.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    # create the subset dataset to load the data you want, and the loader
    finetuning_labels = np.array([self.original_training_set.__getitem__(idx)[2] for idx in finetuning_idxs], dtype=int)
    meta_idxs = [i for i in range(len(finetuning_idxs))]
    random.shuffle(meta_idxs)

    # split the meta idxs in batches
    n_batches = int(np.floor(len(finetuning_idxs)/batch_size))
    meta_idxs_batches = []
    for i in range(n_batches):
      meta_idxs_batches.append(np.array(meta_idxs[batch_size*i:batch_size*(i+1)]))
    meta_idxs_batches.append(np.array(meta_idxs[batch_size*n_batches:]))

    # now fine tune the copied model
    for epoch in range(tuning_epochs):
      for meta_idxs_batch in meta_idxs_batches:
        inputs = mnemonics_to_optimize[0][meta_idxs_batch] # are already in cuda
        labels = finetuning_labels[meta_idxs_batch]
        labels = torch.tensor([self.diz[c] for c in labels])
        labels = labels.to('cuda')
        optimizer.zero_grad()
        outputs = model_copy(inputs)
        labels_encoded = F.one_hot(labels,100).float().cuda()
        loss = self.criterion(outputs, labels_encoded)
        loss.backward()
        optimizer.step()
    

    # UPDATE THE MNEMONICS eq.9/10 ---------------------------------------------
    
    model_copy.eval()
    
    optimizer = optim.SGD(mnemonics_to_optimize, lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
    
    exlvl_training = Subset(self.original_training_set, training_idxs)
    exlvl_loader = DataLoader(exlvl_training, batch_size=batch_size, shuffle=True, num_workers=4)

    for epoch in range(updating_epochs):
      for _, inputs, labels in exlvl_loader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        out_features = model_copy.features(inputs)
        print(out_features.size())
        # compute features mean of mnemonics
        # DA FINIRE DI RIPORTARE... e capire cosa sono per yoyo i logits
        #...

  def exemplar_level_optimization(self, m, task_num, current_task_indices):  
    
    # UPDATING NEW EXEMPLAR-----------------------------------------------------

    # isola gli indici dei nuovi exemplars
    new_exemplar_idxs = []
    for idxs in self.exemplar_sets_idxs[-10:]:
      new_exemplar_idxs += idxs

    # ora ottieni gli mnemonics che poi sono da ottimizzare
    new_mnemonics_data = torch.zeros((10*m, 3, 32, 32))

    for i, idx in enumerate(new_exemplar_idxs):
      new_mnemonics_data[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]

    new_mnemonics = nn.ParameterList()
    new_mnemonics.append(nn.Parameter(new_mnemonics_data))
    new_mnemonics.to('cuda')
    
    print(new_mnemonics[0][0])

    options_new ={'finetuning_idxs': new_exemplar_idxs, 
                  'training_idxs': current_task_indices, 
                  'mnemonics_to_optimize':  new_mnemonics,  
                  'batch_size':64}

    print('---start mnemonics updating---')

    self.update_params(**options_new)    

    print(new_mnemonics[0][0])
    
    # UPDATING OLD EXEMPLARS ---------------------------------------------------

    if task_num:
      # decidi quanti elementi ha ogni exemlar set in a e in b a seconda se m è 
      # pari o dispari
      if m%2:
        l_a = int((m+1)/2)
      else:
        l_a = int(m/2)
      l_b = int(m-l_a)

      # isola gli indici dei vecchi exemplars, dividendoli in due parti
      # ogni classe deve avere circa la metà degli exemplar originali
      old_exemplar_idxs_a = []
      old_exemplar_idxs_b = []
      
      for idxs in self.exemplar_sets_idxs[:-10]:
        old_exemplar_idxs_a += idxs[:l_a]
        old_exemplar_idxs_b += idxs[l_a:]

      old_mnemonics_data_a = torch.zeros((task_num*10*l_a, 3, 32, 32))
      old_mnemonics_data_b = torch.zeros((task_num*10*l_b, 3, 32, 32))

      for i, idx in enumerate(old_exemplar_sets_idxs_a):
        old_mnemonics_data_a[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]
      
      for i, idx in enumerate(old_exemplar_sets_idxs_b):
          old_mnemonics_data_b[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]
      
      old_mnemonics_a = nn.ParameterList()
      old_mnemonics_a.append(nn.Parameter(old_mnemonics_data_a))
      old_mnemonics_a.to('cuda')
      old_mnemonics_b = nn.ParameterList()
      old_mnemonics_b.append(nn.Parameter(old_mnemonics_data_b))
      old_mnemonics_b.to('cuda')

      options_old_a = {'finetuning_idxs':old_exemplar_idxs_a, 
                       'training_idxs':old_exemplar_idxs_b, 
                       'mnemonics_to_optimize':old_mnemonics_a, 
                       'batch_size':64}

      options_old_b = {'finetuning_idxs':old_exemplar_idxs_b, 
                       'training_idxs':old_exemplar_idxs_a, 
                       'mnemonics_to_optimize':old_mnemonics_b, 
                       'batch_size':64}

      self.update_params(**options_old_a) 
      self.update_params(**options_old_b)
    
    # STORE NEW EXEMPLARS

  def model_level_optimization(self):
    
    old_model = copy.deepcopy(self.model)
    old_model.eval()
    old_model.to('cuda')
    n_classes = self.classes_seen+self.CLASSES_PER_BATCH
    print(n_classes)
    optimizer = optim.SGD(self.model.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestones, gamma=self.gamma)
    for epoch in tqdm(range(self.numepochs)):
        
      for _, inputs, labels in self.trainloader:
        inputs = inputs.float().cuda()
        labels = torch.tensor([self.diz[c.item()] for c in labels])

        labels=labels.to('cuda')
        optimizer.zero_grad()
        outputs=self.model(inputs)

        labels_encoded = F.one_hot(labels,100).float().cuda() #CAMBIARE ONE_HOT
        
        if self.classes_seen:
          old_target = old_model(inputs).cuda()
          old_target = torch.sigmoid(old_target).cuda()
          
          target = torch.cat((old_target[:,:self.classes_seen], labels_encoded[:, self.classes_seen:]), dim=1)
          loss = self.criterion(outputs, target)
        else:
          loss = self.criterion(outputs,labels_encoded) 

        loss.backward()
        optimizer.step()
      
      scheduler.step()


  def classify_nme(self, input_batch):
    min_distances = float('inf')*torch.ones(len(input_batch)).cuda() # shape: batch_size --> 128
    y_pred = torch.zeros(len(input_batch), dtype=torch.int8).cuda() # shape: batch_size --> 128
    input_features = self.model.features(input_batch) # shape: (batch_size, feature_size) --> (128, 64)

    for i in range(len(self.exemplar_sets_idxs)):
      ex_mean = self.exemplar_means[i,:]

      # compute distances between inputs features and exemplar set means
      pdist = nn.PairwiseDistance(p=2)
      distances = pdist(input_features, ex_mean) # shape: batch_size --> 128

      # update min distancies and predicted labels
      mask = distances < min_distances
      min_distances[mask] = distances[mask]
      y_pred[mask] = self.exemplar_labels[i]

    return y_pred
    

  def get_new_exemplars(self, batch, m):
    loader = torch.utils.data.DataLoader(batch, batch_size=self.batch_size, shuffle=False, num_workers=4)
    features = np.zeros((0,self.feature_size))
    indices = np.zeros((0), dtype=int)

    # disabilitando il calcolo del gradiente, per ogni batch di immagini calcola
    # le features (shape 128,64), normalizza lungo le righe, concatena gli elemnti

    with torch.no_grad():
      for indexes, images, labels in loader:
        images = images.cuda()
        feature = self.model.features(images).data.cpu().numpy() # shape (128, 64)
        feature = normalize(feature, axis=1, norm='l2') # normalizza lungo le righe
        features = np.concatenate((features,feature), axis=0) 
        indices = np.concatenate((indices,indexes), axis=0) # shape 128

    # calcola il valore medio per ogni colonna (ogni feature) e poi normalizza
    # i vettore dei valori medi
    class_mean = np.mean(features, axis=0) # shape 64
    class_mean = class_mean / np.linalg.norm(class_mean) # shape 64

    exemplar_set = []
    exemplar_features = np.zeros((0,self.feature_size)) # shape _, 64

    for k in range(1, int(m)+1):
        S = np.sum(exemplar_features, axis=0)
        phi = features
        mu = class_mean
        mu_p = 1.0 / k * (phi + S)
        mu_p = normalize(mu_p, axis=1, norm='l2')
        i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
        exemplar_set.append(indices[i])
        addfeature =  np.expand_dims(features[i], axis=0)
        exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

        #remove duplicates
        features = np.delete(features, i, 0)
        indices = np.delete(indices, i, 0)
        
    self.exemplar_sets_idxs.append(exemplar_set)
        
  def reduce_old_exemplars(self, m):
    for y, P_y in enumerate(self.exemplar_sets_idxs):
      self.exemplar_sets_idxs[y] = P_y[:int(m)]


  def __accuracy_fc(self, dl, mapper):
    total = 0.0
    correct = 0.0
    for  _, images, labels in dl:
      labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
      labels = labels.cuda()
      images = images.cuda()
      outputs = self.model(images)
      _, preds = torch.max(outputs, dim=1)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

    acc = correct / total
    return acc


  def __accuracy_nme(self, dl):
    
    total = 0.0
    correct = 0.0
    
    for  _, images, labels in dl:
      labels = labels.cuda()
      images = images.cuda()
      preds = self.classify_nme(images)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

      if self.last_test:
        self.y_pred += preds.tolist()
        self.y_test += labels.tolist()

    acc = correct / total
    return acc


  def plot_confusion_matrix(self):
 
    cm = confusion_matrix(self.y_test, self.y_pred)
    cm = np.log(cm+1)
    fig, ax = plt.subplots(figsize=(7,7))
    sns.heatmap(cm, square=True, cbar=False, ax=ax, cmap=plt.get_cmap('seismic'))
    ax.set_xticks(np.linspace(19,99,5))
    ax.set_yticks(np.linspace(19,99,5))
    ax.set_xticklabels([20,40,60,80,100], rotation=0)
    ax.set_yticklabels([20,40,60,80,100], rotation=0)
    ax.set_title("iCaRL")
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    plt.savefig("iCaRL_"+str(self.randomseed)+"_cm.png")
    plt.show()
    return cm


  def trainer(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    batches=self.original_training_set.getbatches()
    current_test_indexes=[]
    test_acc = []
    self.last_test = False

    for i in range(self.NUM_BATCHES):

      if i == self.NUM_BATCHES-1:
        self.last_test = True

      for exemplar_set in self.exemplar_sets_idxs:
        train_indices[i]=np.concatenate([train_indices[i], np.array(exemplar_set)])

      train_dataset = Subset(self.original_training_set, train_indices[i])
      current_test_indexes += test_indices[i].tolist()
      test_dataset = Subset(self.original_test_set,current_test_indexes)
      self.trainloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)        
      self.model.train()
      self.model_level_optimization()    
      self.classes_seen += 10
      self.model.eval() # Set Network to evaluation mode

      # update exemplars number
      m=int(2000/(int(i*10+10)))

      # reduce the number of each exemplars set
      self.reduce_old_exemplars(m) 
      
      for classlabel in batches[i]:
        indexes_class = self.original_training_set.get_class_indexes(classlabel)
        current_class = Subset(self.original_training_set, indexes_class)
        self.get_new_exemplars(current_class, m)
      
      self.exemplar_level_optimization(m, i, train_indices[i])

      # compute means of exemplar set
      # cycle for each exemplar set
      self.exemplar_means = torch.zeros((0, self.feature_size), dtype=torch.float).cuda()
      self.exemplar_labels = []
      for i in range(len(self.exemplar_sets_idxs)):
        exemplars_dataset = Subset(self.original_training_set, self.exemplar_sets_idxs[i])
        exemplars_loader = torch.utils.data.DataLoader(exemplars_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)
        ex_features = torch.zeros((0, self.feature_size), dtype=torch.float).cuda() # alla fine shape: (len(exemplar_set), feature_size) --> (m, 64)
      
        with torch.no_grad():
          _, _, exemplar_label = self.original_training_set.__getitem__(self.exemplar_sets_idxs[i][0]) 
          self.exemplar_labels.append(exemplar_label)
          # cycle for each batch in the current exemplar set
          for _,  exemplars, _ in exemplars_loader:
          
            # get exemplars features
            exemplars = exemplars.cuda()
            features = self.model.features(exemplars) # shape: (len(exemplars), feature_size)
          
            # normalize 
            feature_norms = torch.norm(features, p=2, dim=1) # shape: len(exemplars)
            feature_norms.unsqueeze_(1) # shape: (len(exemplars), 1)
            features = features/feature_norms
          
            # concatenate over columns
            ex_features = torch.cat((ex_features, features), dim=0)
          
        # compute current exemplar set mean and normalize it
        ex_mean = torch.mean(ex_features, dim=0) # shape: feature_size --> 64
        ex_mean = ex_mean/torch.norm(ex_mean)
        ex_mean.unsqueeze_(0) # shape: (1, feature_size) --> (1, 64)
        self.exemplar_means = torch.cat((self.exemplar_means, ex_mean), dim=0) # shape: (n_examplar sets, feature size)
      

      print('accuracy on training set:', 100*self.__accuracy_fc(self.trainloader,self.diz))
      # print('accuracy on test set:', self.__accuracy_on(self.testloader,self,self.diz))
      current_test_acc = self.__accuracy_nme(self.testloader)
      print('accuracy on test set:', 100*current_test_acc)
      print('-' * 80)
      test_acc.append(current_test_acc)

    # compute comfusion matrix and save results
    cm = self.plot_confusion_matrix()
    with open('iCaRL_'+str(self.randomseed)+"_cm", 'wb') as file:
      pickle.dump(cm, file, protocol=pickle.HIGHEST_PROTOCOL)
    with open('iCaRL_'+str(self.randomseed)+"_testacc", 'wb') as file:
      pickle.dump(test_acc, file, protocol=pickle.HIGHEST_PROTOCOL)


In [29]:
method = mnemonics(randomseed=203)
method.trainer()

Files already downloaded and verified
Files already downloaded and verified
10


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


tensor([[[-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         ...,
         [-0.1658, -0.3564,  0.1860,  ..., -0.3271, -1.8957, -1.8957],
         [-0.1365, -0.3857,  0.1274,  ..., -0.4590, -1.8957, -1.8957],
         [-0.2391, -0.3857,  0.0541,  ..., -0.6643, -1.8957, -1.8957]],

        [[-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         ...,
         [-1.4847, -1.3165, -0.7814,  ..., -0.5673, -1.8975, -1.8975],
         [-1.3929, -1.3318, -0.7967,  ..., -0.6285, -1.8975, -1.8975],
         [-1.4082, -1.3471, -0.8731,  ..., -0.7967, -1.8975, -1.8975]],

        [[-1.5965, -1.5965, -1.5965,  ..., -1.5965, -1.5965, -1.5965],
         [-1.5965, -1.5965, -1.5965,  ..., -1

KeyboardInterrupt: ignored

In [34]:
np.floor(4/3)

1.0

In [15]:
a = []
b = [1,2,3]
a.append(b)
a

[[1, 2, 3]]